In [ ]:
!nvidia-smi

Sat Dec  4 05:37:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import os
#Edit this to match your choice of main_path
main_path = "/content/drive/My Drive/11777_multimodal_course"
os.chdir(main_path)
#!pip --version
os.chdir("final_project/seq2seq_data")

Mounted at /content/drive


# Data Preprocessing for HF Training

In [ ]:
import json

def convert_to_HF_format(story_file,caption_file,output_file):
    story_f = open(story_file)
    caption_f = open(caption_file)
    story_json = json.loads(story_f.read())
    caption_json = json.loads(caption_f.read())
    output_json = []
    for k,v in story_json.items():
        output_dict = {}
        story_id = v["id"]
        album_id = v["album_id"]
        story = ' '.join(v["sent_texts"])
        captions = []
        for img_id in v["img_ids"]:
            caption = caption_json[img_id][0][0].strip()
            captions.append(caption)
        output_dict['input'] = ' \n '.join(captions)
        output_dict['output'] = story
        output_dict['album_id'] = album_id
        output_dict['story_id'] = story_id
        output_json.append(output_dict)
    print(len(output_json))
    print(output_json[:10])
    with open(output_file,'w') as out_f:
        json.dump(output_json,out_f,indent=4)
    out_f.close()
    print(f"Output JSON in HF format written to file {output_file}")

In [ ]:
story_file = 'stories_train.json'
caption_file = 'captions/captions_train_sample100.json'
output_file = 'training/stories_train_HF.json'
#convert_to_HF_format(story_file,caption_file,output_file)

# Get BART and T5 Tokenizer Lengths of the New Inputs

In [ ]:
!pip install datasets
!pip install --no-cache-dir transformers==4.3.3 sentencepiece==0.1.95

In [ ]:
#this cell contains the filenames and so forth to be used by most of the code below. for convenience purposes they are all here (for quick and efficient modification)

train_json_filename = 'training/stories_train_HF_kw_5grams.json'

In [ ]:
import json

# load data from json file. format: list of dictionaries, each containing one example, where keys of each dict are "document", "edited_key" (specified as argument), "id", and "summary" (and possibly additional keys, these are just the mandatory ones)
def load_from_json(filename):
    with open(filename) as f:
        loaded_json = json.loads(f.read())
    print(type(loaded_json))
    print(loaded_json[0])
    inputs = [x['input'] for x in loaded_json]
    outputs = [x['output'] for x in loaded_json]
    print('input:', inputs[0])
    print('output:', outputs[0])
    return loaded_json

loaded_train_json = load_from_json(train_json_filename)

<class 'list'>
{'input': 'tree <s> big wall with a grassy \n plaque covered with information displayed <s> tree \n branches wrapped around tree trunk \n man standing in front <s> tree \n parklet and dirt path <s> tree', 'output': 'we found this tree when we were walking in a nearby town . it turns out it is a popular attraction here . the tree is very unusual , with its roots exposed . the trunk was really wide , as much as 12 feet ! you can see how big these roots are - pretty amazing !', 'album_id': '72157605930515606', 'story_id': '30356'}
input: tree <s> big wall with a grassy 
 plaque covered with information displayed <s> tree 
 branches wrapped around tree trunk 
 man standing in front <s> tree 
 parklet and dirt path <s> tree
output: we found this tree when we were walking in a nearby town . it turns out it is a popular attraction here . the tree is very unusual , with its roots exposed . the trunk was really wide , as much as 12 feet ! you can see how big these roots are - pre

In [ ]:
#modified : this cell has been inserted

#convert loaded json format to format required to load into huggingface dataset. edited_key argument same as earlier
def convert_json_format(loaded_json):#(loaded_train_json,loaded_val_json):
    loaded_dict = {}
    input_lst = [x['input'] for x in loaded_json]
    print(len(input_lst))
    output_lst = [x['output'] for x in loaded_json]
    print(len(output_lst))
    loaded_dict['input'] = input_lst
    loaded_dict['output'] = output_lst
    print(loaded_dict['input'][:3])
    print(loaded_dict['output'][:3])
    return loaded_dict
      
#converted_commongen_json = convert_json_format(loaded_train_json,loaded_val_json) #67389 train examples, 4018 val examples
converted_train_json = convert_json_format(loaded_train_json)

10000
10000
['tree <s> big wall with a grassy \n plaque covered with information displayed <s> tree \n branches wrapped around tree trunk \n man standing in front <s> tree \n parklet and dirt path <s> tree', 'tree <s> big wall with a grassy \n plaque covered with information displayed <s> tree \n branches wrapped around tree trunk \n man standing in front <s> tree \n parklet and dirt path <s> tree', 'tree <s> big wall with a grassy \n plaque covered with information displayed <s> tree \n branches wrapped around tree trunk \n man standing in front <s> tree \n parklet and dirt path <s> tree']
['we found this tree when we were walking in a nearby town . it turns out it is a popular attraction here . the tree is very unusual , with its roots exposed . the trunk was really wide , as much as 12 feet ! you can see how big these roots are - pretty amazing !', 'they went to the botanic gardens specifically to see the large tree . there was an informational sign posted near it . the roots were h

In [ ]:
#modified : this cell has been inserted

from datasets import Dataset

#convert the converted_json to huggingface dataset format
def load_huggingface_dataset_from_dict(converted_json):
    #train_dataset = Dataset.from_dict(converted_json["train"])
    #val_dataset = Dataset.from_dict(converted_json["validation"])
    loaded_hf_dataset = Dataset.from_dict(converted_json)
    print(type(loaded_hf_dataset))
    print(loaded_hf_dataset)
    return loaded_hf_dataset

commongen_train = load_huggingface_dataset_from_dict(converted_train_json)
print(commongen_train[:5])

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['input', 'output'],
    num_rows: 10000
})
{'input': ['tree <s> big wall with a grassy \n plaque covered with information displayed <s> tree \n branches wrapped around tree trunk \n man standing in front <s> tree \n parklet and dirt path <s> tree', 'tree <s> big wall with a grassy \n plaque covered with information displayed <s> tree \n branches wrapped around tree trunk \n man standing in front <s> tree \n parklet and dirt path <s> tree', 'tree <s> big wall with a grassy \n plaque covered with information displayed <s> tree \n branches wrapped around tree trunk \n man standing in front <s> tree \n parklet and dirt path <s> tree', 'statue of the scottish union <s> walkway \n stone clock tower <s> building \n large <s> pink <s> purple flowers and white leaves \n faces <s> large display of different sculpture \n white <s> food <s> pie topped with gravy', 'statue of the scottish union <s> walkway \n stone clock tower <s> bu

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM                   
from transformers import Trainer, TrainingArguments

# see https://discuss.huggingface.co/t/error-with-new-tokenizers-urgent/2847/4
bart_or_t5 = "t5"
if bart_or_t5 == "bart":
    tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base", use_fast=False)
if bart_or_t5 == "t5":
    tokenizer = AutoTokenizer.from_pretrained("t5-base", use_fast=False)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [ ]:
#test0 = tokenizer.tokenize(loaded_test_json[0]["input"])
test1 = tokenizer.tokenize("hello <s> bye")
test2 = tokenizer.tokenize("hello <sep> bye")
test3 = tokenizer.tokenize("hello _eos bye")
test4 = tokenizer.tokenize("hello \n bye")
#print(test0)
print(test1)
print(test2)
print(test3)
print(test4)

['▁hello', '▁', '<', 's', '>', '▁by', 'e']
['▁hello', '▁', '<', 's', 'e', 'p', '>', '▁by', 'e']
['▁hello', '▁', '_', 'e', 'o', 's', '▁by', 'e']
['▁hello', '▁by', 'e']


In [ ]:
#test0_n = tokenizer.convert_tokens_to_string(test0)
test1_n = tokenizer.convert_tokens_to_string(test1)
test2_n = tokenizer.convert_tokens_to_string(test2)
test3_n = tokenizer.convert_tokens_to_string(test3)
test4_n = tokenizer.convert_tokens_to_string(test4)

#print(test0_n)
print(test1_n)
print(test2_n)
print(test3_n)
print(test4_n)

hello <s> bye
hello <sep> bye
hello _eos bye
hello bye


In [ ]:
def map_to_length(x):
    x["article_len"] = len(tokenizer(x["input"]).input_ids)
    x["article_longer_32"] = int(x["article_len"] > 32)
    x["article_longer_64"] = int(x["article_len"] > 64)
    x["article_longer_128"] = int(x["article_len"] > 128)
    x["article_longer_256"] = int(x["article_len"] > 256)
    x["article_longer_512"] = int(x["article_len"] > 512)
    x["article_longer_1024"] = int(x["article_len"] > 1024)
    x["summary_len"] = len(tokenizer(x["output"]).input_ids)
    x["summary_longer_32"] = int(x["summary_len"] > 32)
    x["summary_longer_64"] = int(x["summary_len"] > 64)
    x["summary_longer_128"] = int(x["summary_len"] > 128)
    x["summary_longer_256"] = int(x["summary_len"] > 256)
    return x

def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print("sample_size: ", sample_size)
    print(
        "Article Mean: {}, %-Articles > 32: {}, %-Articles > 64: {}, %-Articles > 128: {}, %-Articles > 256: {}, %-Articles > 512: {}, %-Articles > 1024: {}, Summary Mean: {}, %-Summary > 32: {}, %-Summary > 64: {}, %-Summary > 128: {}, %-Summary > 256: {}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_32"]) / sample_size, 
            sum(x["article_longer_64"]) / sample_size, 
            sum(x["article_longer_128"]) / sample_size, 
            sum(x["article_longer_256"]) / sample_size, 
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["article_longer_1024"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_32"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
            sum(x["summary_longer_256"]) / sample_size,
        )
    )

In [ ]:
sample_size = len(commongen_train)
train_data_stats = commongen_train.select(range(len(commongen_train))).map(map_to_length, num_proc=1)

  0%|          | 0/10000 [00:00<?, ?ex/s]

In [ ]:
output = train_data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)

#T5-base:
#train full captions: 
#Article Mean: 66.8844, %-Articles > 32: 1.0, %-Articles > 64: 0.6227, %-Articles > 128: 0.0, %-Articles > 256: 0.0, %-Articles > 512: 0.0, %-Articles > 1024: 0.0, Summary Mean: 73.893, %-Summary > 32: 0.9863, %-Summary > 64: 0.6099, %-Summary > 128: 0.0317, %-Summary > 256: 0.0002
#keyphrases 1-grams:
#Article Mean: 112.5017, %-Articles > 32: 1.0, %-Articles > 64: 0.9997, %-Articles > 128: 0.1463, %-Articles > 256: 0.0, %-Articles > 512: 0.0, %-Articles > 1024: 0.0, Summary Mean: 73.893, %-Summary > 32: 0.9863, %-Summary > 64: 0.6099, %-Summary > 128: 0.0317, %-Summary > 256: 0.0002
#keyphrases 3-grams:
#Article Mean: 76.7588, %-Articles > 32: 1.0, %-Articles > 64: 0.8285, %-Articles > 128: 0.0003, %-Articles > 256: 0.0, %-Articles > 512: 0.0, %-Articles > 1024: 0.0, Summary Mean: 73.893, %-Summary > 32: 0.9863, %-Summary > 64: 0.6099, %-Summary > 128: 0.0317, %-Summary > 256: 0.0002
#keyphrases 5-grams:
#Article Mean: 49.8232, %-Articles > 32: 0.9755, %-Articles > 64: 0.0628, %-Articles > 128: 0.0, %-Articles > 256: 0.0, %-Articles > 512: 0.0, %-Articles > 1024: 0.0, Summary Mean: 73.893, %-Summary > 32: 0.9863, %-Summary > 64: 0.6099, %-Summary > 128: 0.0317, %-Summary > 256: 0.0002

  0%|          | 0/1 [00:00<?, ?ba/s]

sample_size:  10000
Article Mean: 49.8232, %-Articles > 32: 0.9755, %-Articles > 64: 0.0628, %-Articles > 128: 0.0, %-Articles > 256: 0.0, %-Articles > 512: 0.0, %-Articles > 1024: 0.0, Summary Mean: 73.893, %-Summary > 32: 0.9863, %-Summary > 64: 0.6099, %-Summary > 128: 0.0317, %-Summary > 256: 0.0002


In [ ]:
from tqdm.auto import tqdm

train_article_lens = []
train_summary_lens = []
for x in tqdm(train_data_stats):
    train_article_lens.append(x["article_len"])
    train_summary_lens.append(x["summary_len"])
print(max(train_article_lens))
print(max(train_summary_lens))

#T5-base:
#train full captions:
#95 (choose 128 max encoder len)
#374 (choose 128 max decoder len)
#keyphrases 1-grams:
#163 (choose 160 max encoder len)
#374 (choose 128 max decoder len)
#keyphrases 3-grams:
#133 (choose 128 max encoder len)
#374 (choose 128 max decoder len)
#keyphrases 5-grams:
#87 (choose 96 max encoder len)
#374 (choose 128 max decoder len)

  0%|          | 0/10000 [00:00<?, ?it/s]

87
374


# Keywords/keyphrases 

In [ ]:
!pip install git+https://github.com/LIAAD/yake

In [ ]:
'''default settings (used for most experiments):
language = "en"
max_ngram_size = 5 #2, 3, or 5 (for everything other than T5-large (where 3 is optimal), 2 is optimal)
deduplication_threshold = 0 #set to 0 for maximum deduplication (no overlaps)
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 10
'''

import yake
#kw_extractor = yake.KeywordExtractor()
language = "en"
max_ngram_size = 5
deduplication_threshold = 0.1 #between 0 and 1 inclusive. set to 0 for maximum deduplication (no overlaps).
    #seems 0.5 is decent for max_ngram_size = 1
    #seems 0.2 is decent for max_ngram_size = 3
    #seems 0.1 is decent for max_ngram_size = 5
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 10

kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)

In [ ]:
keywords = kw_extractor.extract_keywords("hello my name is blah")
print(keywords)

[('blah', 0.15831692877998726)]


In [ ]:
import string

def extract_keywords_yake(example):
    #print(example)
    keywords = kw_extractor.extract_keywords(example)
    keywords_only = [x[0].lower() for x in keywords]
    #print(keywords_only)
    try:
        indices_lst = [example.translate(str.maketrans('', '', string.punctuation)).lower().split(' ').index(x.translate(str.maketrans('', '', string.punctuation)).lower().split(' ')[0]) for x in keywords_only]
        sorted_indices, sorted_keywords = zip(*sorted(zip(indices_lst, keywords_only)))
    except:
        #print(example.translate(str.maketrans('', '', string.punctuation)).lower().split(' '))
        #print([x.translate(str.maketrans('', '', string.punctuation)).lower().split(' ')[0] for x in keywords_only])
        #print("error: {} | {}".format(example,keywords_only))
        sorted_keywords = keywords_only
    #print("keywords_only: ", keywords_only)
    keywords_str = ' <s> '.join(sorted_keywords)
    return keywords_str

In [ ]:
from tqdm import tqdm

def get_caption_keywords(normal_HF_file, output_file):
    with open(normal_HF_file) as input_f:
        input_json = json.loads(input_f.read())
    print(len(input_json))
    output_json = []
    for example in tqdm(input_json):
        output_dict = {}
        captions = example['input'].split(' \n ')
        keyword_lst = []
        for caption in captions:
            keyword_lst.append(extract_keywords_yake(caption))
        output_dict['input'] = ' \n '.join(keyword_lst)
        output_dict['output'] = example['output']
        output_dict['album_id'] = example['album_id']
        output_dict['story_id'] = example['story_id']
        output_json.append(output_dict)
    print(len(output_json))
    print(output_json[:2])
    with open(output_file,'w') as out_f:
        json.dump(output_json,out_f,indent=4)
    out_f.close()
    print(f"Output json written to {output_file}")

In [ ]:
normal_HF_file = 'training/stories_test_HF.json'
output_file = 'training/stories_test_HF_kw_5grams.json'
#get_caption_keywords(normal_HF_file, output_file)

# Summarization - Pegasus

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

In [ ]:
#Example
ARTICLE_TO_SUMMARIZE = ("PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions.\
The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the \
shutoffs which were expected to last through at least midday tomorrow.")
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

summary_ids = model.generate(inputs['input_ids'])
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1648: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


['Images courtesy of AFP, EPA, Getty Images and Reuters']


In [ ]:
from tqdm import tqdm

def summarize_captions(normal_HF_file, output_file):
    with open(normal_HF_file) as input_f:
        input_json = json.loads(input_f.read())
    print(len(input_json))
    output_json = []
    counter = 0
    for example in tqdm(input_json):
        output_dict = {}
        captions = example['input'].replace(' \n ',' ')
        inputs = tokenizer([captions], max_length=1024, truncation=True, return_tensors='pt')
        summary_ids = model.generate(inputs['input_ids'])
        summary_output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]     
        if counter < 50:
            print(captions)
            print(summary_output)   
        output_dict['input'] = summary_output
        output_dict['output'] = example['output']
        output_dict['album_id'] = example['album_id']
        output_dict['story_id'] = example['story_id']
        output_json.append(output_dict)
        counter += 1
    print(len(output_json))
    print(output_json[:2])
    with open(output_file,'w') as out_f:
        json.dump(output_json,out_f,indent=4)
    out_f.close()
    print(f"Output json written to {output_file}")

In [ ]:
normal_HF_file = 'training/stories_val_HF.json'
output_file = 'training/stories_val_HF_summarized.json'
#summarize_captions(normal_HF_file, output_file)